In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import pickle as pkl

In [2]:
file = open('/kaggle/input/main-twitter/vocabulary.pkl', "rb")
vocabulary = pkl.load(file)
file.close()

In [3]:
df = pd.read_csv('/kaggle/input/main-twitter/labeled.csv')

In [4]:
df = df[df['class'] != '-']
df.drop(['name'], axis=1, inplace=True)
df.dropna(how='any', inplace=True)
df.reset_index(drop=True, inplace=True)
df['class'] = df['class'].astype(int)
df = df[df['class'] >= 0]

In [5]:
def isInVocabulary(word, voc=vocabulary):
    """
    Searchs for word from dictonary based on first letter
    """
    first_letter = word[0]
    if first_letter not in voc:
        return False
    if word in voc[first_letter]:
        return True
    else:
        return False

In [6]:
def stem(string):
    l=[]
    vowels=["a","ı","o","u","e","ə","i"]
    string=string.split()
    for i in string:
        if i.isupper() or (string.index(i)!=0 and i[0].isupper()):
            # xüsusi isimlər və abbr. üçün
            l.append(i)
        else:
            for j in range(len(i),0,-1):
                if isInVocabulary(i[:j].casefold()): # i[:j].casefold() in words:
                    # adi şəkilçilər üçün
                    l.append(i[:j])
                    break
                elif i[j-1] in vowels and (i[j-2]=="y" or i[j-2]=="ğ") :
                    # bitişdirici samitlər üçün
                    if isInVocabulary((i[:j-2]+"k").casefold()): # (i[:j-2]+"k").casefold() in words:
                        l.append(i[:j-2]+"k")
                        break
                    elif isInVocabulary((i[:j-2]+"q").casefold()): # (i[:j-2]+"q").casefold() in words:
                        l.append(i[:j-2]+"q")
                        break
    return ' '.join(l)

In [7]:
df['Text'] = df['Text'].apply(lambda row: stem(row))

In [8]:
stemmed_data = df.copy()
stemmed_data.to_csv('clean_stemmed_data.csv', index=False)

In [11]:
!pip install lazy-text-predict

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 3.5 MB/s eta 0:00:00a 0:00:010m


In [9]:
from warnings import filterwarnings
filterwarnings('ignore')

In [12]:
from lazytextpredict import basic_classification

trial=basic_classification.LTP(Xdata=list(df["Text"].values),Ydata=list(df["class"].values), x_col='Text', y_col='class', models='all') 
# Xdata is a list of text entries, and Ydata is a list of corresponding labels.
# csv and xlsx give options to load data from those file formats (you can pass the file or the file's location)
# x_col and y_col are strings that specify the columns of the # text and label columns in your csv or xlsx file respectively.
# You can choose between 'transformers'-based, 'count-vectorizer'-based, and 'all' models.

trial.run(training_epochs=5) 
#This trains the models specified above on the data you loaded. 
#Here you can specify the number of training epochs. 
#Fewer training epochs will give poorer performance, but will run quicker to allow debugging.

trial.print_metrics_table()

X_train length: 8812
X_test length: 980
Y_train length: 8812
Y_test length: 980
Training on a dataset with 3 labels


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

***** Running training *****
  Num examples = 8812
  Num Epochs = 5
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 2755
  Number of trainable parameters = 109484547
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"


...

{'eval_loss': 1.0161653757095337, 'eval_accuracy': 0.5173469387755102, 'eval_f1': 0.35278399187516296, 'eval_precision': 0.2676478550603915, 'eval_recall': 0.5173469387755102, 'eval_full_report': {'0': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 185}, '1': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 288}, '2': {'precision': 0.5173469387755102, 'recall': 1.0, 'f1-score': 0.6819098856758574, 'support': 507}, 'accuracy': 0.5173469387755102, 'macro avg': {'precision': 0.17244897959183672, 'recall': 0.3333333333333333, 'f1-score': 0.2273032952252858, 'support': 980}, 'weighted avg': {'precision': 0.2676478550603915, 'recall': 0.5173469387755102, 'f1-score': 0.35278399187516296, 'support': 980}}, 'eval_runtime': 16.7579, 'eval_samples_per_second': 58.48, 'eval_steps_per_second': 0.955, 'epoch': 5.0}


PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


Training on a dataset with 3 labels
ERROR
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


Training on a dataset with 3 labels
ERROR
best parameters are:
{'clf__alpha': 0.01, 'clf__fit_prior': True, 'tfidf__use_idf': False, 'vect__ngram_range': (1, 2)}
{'eval_loss': 0.2510204081632653, 'eval_accuracy': 0.7489795918367347, 'eval_f1': 0.7302273673540053, 'eval_precision': 0.7406893056889793, 'eval_recall': 0.7221644721644721, 'eval_full_report': '              precision    recall  f1-score   support\n\n           0       0.71      0.62      0.66       185\n           1       0.75      0.75      0.75       288\n           2       0.76      0.79      0.78       507\n\n    accuracy                           0.75       980\n   macro avg       0.74      0.72      0.73       980\nweighted avg       0.75      0.75      0.75       980\n'}
                    Model            loss        accuracy              f1       precision          recall
        bert-base-uncased         0.89947         0.71837         0.71734         0.71715         0.71837
           albert-base-v2          1.0